<p style="font-weight:bold; font-size: 40px; color:#0171ff; padding-top: 50px; text-align:center"> How to compute Sensitivities </p>

The International Financial Reporting Standard (IFRS) 17 requires entities to disclose a Sensitivity Analysis (SA) to report the effects of reasonables changes in the assumptions used to create Cashflows.

The Best Estimate (BE) is the Cashflow created on the set of parameters believed to be most accurate, given hystorical data and state-of-the-art predictions.

Nevertheless, the variation of these parameters must be evaluated for the reporting period on measurement components presented in previous Present Value Series, like CSM LC and Financial Performance. 

A list of common Scenario includes but is not exhausted by:
1. Increase/decrease of Yield Curves (YC),
2. Increase/decrease of Mortality, Longevity or Disability,
3. Equity/Property fluctuation,
4. Liabilities on Incurred Claims (LIC).
   
> In addition, entities can introduce new Scenarios when thought to be relevant for the regulator.

The aim of this notebook is to illustrate the IFRS 17 *Sensitivities* calculation using Systemorph Cloud Technology: it enables our partners to perform a complete set of SA at the simple cost of producing new inputs for each Scenario they are interested. Currently, Cashflow, Actuals and Yield Curves inputs are designed to depends on Scenarios: if it is not specified, the platform will assumes that they belong to BE.

The company portfolio structure, i.e. **GIC** hierarchy, is not meant to change because of external variables and therefore does not support SA.



# Scenario Definition

Scenarios are identified as **dimension** and imported via the *dimensions.csv* file. As BE is the default Scenario, it does not have to be specified. If a given Scenario is not been imported, all attempts to upload related Data will fail. 

The effect of the Scenario is parametrized in IFRS17 by a change in the value of some AoCSteps. In order to save time and efford, the Platform implemetes the following business rules:
1. if an AoCStep is present, it will be saved and used for calculations;
2. if an AoCStep is Zero, it will be save instead of be discarded (unlike in BE import);
3. if an AoCStep is absent, the BE value is used.
4. if an AoCStep is absent in either Scenario and BE case, a Zero will be used.

These rules minimize the amount of Data needed to performe a SA, since the Platform will retrieve all unchanged AoCSteps from the DataBase. If the AoCSteps available does not suits the needs of the business Case, new ones can be introduced in thee *dimensions.csv*.

Even if a dimension import can be performed several times with different files, these fundamental data structures have such deep consequences on calculation flow and numerical results that it is strongly recommended to not change them after the import of Transactiona Data (TD, i.e. Cashflows and Actuals).

In [0]:
#!eval-notebook "Initialization"

# BE Data Import

Since BE is the default case no other Data are available, and the YC must be imported *before* the TD.

In [0]:
await Import.FromFile("CF_G_2023_1.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

# Scenario Data Import

Importing Scenario TD does not needs more steps that importing a BE TD. The information about the Scenario is saved in the Main tab, while the perturbed values are in the Cashflow tab (download the Exce file to compare).

In our example, the mortality increase has been modeled with a change in Assumption Update (AU) and Combined Liability (CL) with AmoutType Claims.

## Transactional Data

In [0]:
await Import.FromFile("CF_G_2023_1_MTUP.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("CF_G_2023_1_MTDN.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

## Interaction between Yield Curve and Transactional Data

Systemorph Cloud Technology interpretes SA as a direct perturbation of BE Data, either via YC or TD, not both. Therefore, it allows Scenario TD to be discounted only using BE YC and Scenario YC discouting BE TD, respectively. 

# Report

In [0]:
var pv = ifrs17.PresentValues;
pv.ReportingNode = "G";
pv.ReportingPeriod = (2023, 1);
pv.ColumnSlices = new[] {"EstimateType"};
pv.DataFilter = new[] {("EconomicBasis", "L")};
await pv.ToReportAsync

In [0]:
pv.Scenario = "MTUP10pct";
await pv.ToReportAsync

In [0]:
pv.Scenario = "All";
await pv.ToReportAsync

In [0]:
pv.Scenario = "Delta";
await pv.ToReportAsync

In [0]:
var allocatedTechnicalMargins = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMargins.ReportingNode = "G";
allocatedTechnicalMargins.ReportingPeriod = (2023, 1);
allocatedTechnicalMargins.DataFilter = new [] {("GroupOfContract", "GIC1")};
allocatedTechnicalMargins.Scenario = null;
(await allocatedTechnicalMargins.ToReportAsync)

In [0]:
var allocatedTechnicalMargins = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMargins.ReportingNode = "G";
allocatedTechnicalMargins.ReportingPeriod = (2023, 1);
allocatedTechnicalMargins.DataFilter = new [] {("GroupOfContract", "GIC1")};
allocatedTechnicalMargins.Scenario = "All";
(await allocatedTechnicalMargins.ToReportAsync)